In [8]:
from recommendation_engine import Movie_recomendation_system

In [9]:
%%time
Movie_recomendation_system.automatic_recommendation(1)

Wall time: 2.92 s


['Trainspotting (1996)',
 'Dead Poets Society (1989)',
 'While You Were Sleeping (1995)',
 'First Wives Club, The (1996)',
 'Right Stuff, The (1983)']

In [10]:
%%time
Movie_recomendation_system.automatic_recommendation(1)

Wall time: 3 s


['Big Night (1996)',
 'Dragonheart (1996)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Godfather, The (1972)',
 'Beauty and the Beast (1991)',
 'Scream (1996)']

In [11]:
%%time
Movie_recomendation_system.automatic_recommendation(1)

Wall time: 3.05 s


['Big Night (1996)',
 'Dragonheart (1996)',
 'Glory (1989)',
 'Courage Under Fire (1996)',
 'Fried Green Tomatoes (1991)',
 'Volcano (1997)',
 'Executive Decision (1996)']

In [5]:
%%time
Movie_recomendation_system.automatic_recommendation(1)

Wall time: 2.86 s


['Die Hard (1988)',
 'Dave (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Stand by Me (1986)',
 'Star Trek: First Contact (1996)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars (1977)',
 'Get Shorty (1995)',
 'Shawshank Redemption, The (1994)',
 'First Wives Club, The (1996)',
 'Quiz Show (1994)',
 'Abyss, The (1989)',
 'Raising Arizona (1987)',
 'Silence of the Lambs, The (1991)',
 'Toy Story (1995)']

In [6]:
%%time
Movie_recomendation_system.automatic_recommendation(1)

Wall time: 2.84 s


['Apocalypse Now (1979)',
 '2001: A Space Odyssey (1968)',
 "Monty Python's Life of Brian (1979)",
 'True Lies (1994)',
 'Die Hard (1988)',
 'Dave (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Stand by Me (1986)',
 'Star Trek: First Contact (1996)',
 'North by Northwest (1959)',
 'Face/Off (1997)',
 'Return of the Jedi (1983)',
 'Seven (Se7en) (1995)',
 'Twelve Monkeys (1995)',
 'Toy Story (1995)',
 'Right Stuff, The (1983)']

In [7]:
%%time
Movie_recomendation_system.automatic_recommendation(1)

Wall time: 2.98 s


['North by Northwest (1959)',
 'Face/Off (1997)',
 'Seven (Se7en) (1995)',
 'Unforgiven (1992)',
 'Get Shorty (1995)',
 'Raiders of the Lost Ark (1981)']